In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import re
from tqdm import tqdm

In [ ]:
messages_filepath, categories_filepath, database_filepath = "data/disaster_messages.csv", "data/disaster_categories.csv", "data/DisasterResponse.db"

In [75]:
def load_data(messages_filepath, categories_filepath):
    """
    Loads data from the parameters and merge them in a dataframe

    Parameters:
    messages_filepath (string): filepath to the file containing messages
    categories_filepath (string): filepath to the file containing categories

    Returns:
    df (dataframe): dataframe containing the loaded and merged information
    """

    df_messages = pd.read_csv(messages_filepath)
    df_messages.drop('original', inplace=True, axis=1)
    df_messages.set_index('id', inplace=True)
    
    df_categories = pd.read_csv(categories_filepath)
    df_categories.set_index('id', inplace=True)

    df = pd.concat([df_messages, df_categories], axis=1)

    return df

def clean_data(df):
    """
    Cleans and transform data from the provided dataframe, transforming the 'categories' column to a multi-column values

    Parameters:
    df (dataframe): Dataframe containing messages and categories

    Returns:
    df (dataframe): Dataframe cleaned and transformed
    """

    # split the 'categories' column in multi-columns one-hot encoded
    # print(df)
    categories = df['categories'].tolist()

    df_categories = pd.DataFrame([sub.split(";") for sub in categories])
    categories = df_categories.iloc[1, :].str.split("-").tolist()
    categories = [i[0] for i in categories]
    
    df_categories.columns = categories
    for category in tqdm(categories):
        values = df_categories[category].str.split("-").tolist()
        df_categories[category] = pd.Series([i[1] for i in values]).astype(bool).astype(int)

    df.drop('categories', axis=1, inplace=True)

    df = pd.concat([df, df_categories])
    df.drop_duplicates(keep='first', inplace=True)

    df['message'] = df['message'].str.lower()

    return df


def save_data(df, database_filename):
    """
    Saves the df dataframe into a SQLite database usng the database filename provided

    Parameters:
    df (dataframe): A dataframe containing the information to be stored in the SQLite database
    database_filename (string): The SQLite database filename
    """
    conn = sqlite3.connect(database_filename)
    df.to_sql(database_filename, con = conn, if_exists='replace', index=False)

In [76]:
df = load_data(messages_filepath, categories_filepath)
print('Cleaning data...')

df = clean_data(df)

print('Saving data...\n    DATABASE: {}'.format(database_filepath))
save_data(df, database_filepath)

print('Cleaned data saved to database!')

Cleaning data...
100%|██████████| 36/36 [00:04<00:00,  8.92it/s]
Saving data...
    DATABASE: data/DisasterResponse.db
Cleaned data saved to database!


In [ ]:
df.head()